In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

# Load the Excel file
file_path = 'interopSurvey.xlsx'
df = pd.read_excel(file_path)

# Inspect the first few rows of the DataFrame to understand its structure
df.head()


In [ ]:
# Total number of rows where the first two columns include yes
yes_rows = df[df['Q52'].str.contains('yes', case=False, na=False) & df['Q71'].str.contains('yes', case=False, na=False)]
print(f"Total number of rows where the first two columns include yes: {len(yes_rows)}")

In [ ]:
# check how many rows are not null in column 21
df[df.columns[21]].notnull().sum()


In [ ]:
# Columns T to AE in Excel correspond to indices 19 to 30 in Python (0-based indexing)
columns_of_interest = df.columns[19:31]  # Select columns T to AE

# Count the number of rows where columns T to AE are not all 'n/a'
# We will treat both 'n/a' and NaN as equivalent for this purpose
count_non_na_rows = df[columns_of_interest].replace('n/a', pd.NA).dropna(how='all').shape[0]

count_non_na_rows


In [ ]:
# Columns T to AE in Excel correspond to indices 19 to 30 in Python (0-based indexing)
columns_of_interest = df.columns[20:31]  # Select columns T to AE

# Count the number of rows where columns T to AE are not all 'n/a'
# We will treat both 'n/a' and NaN as equivalent for this purpose
count_non_na_rows = df[columns_of_interest].replace('n/a', pd.NA).dropna(how='all').shape[0]

count_non_na_rows


In [ ]:
# Columns T to AE in Excel correspond to indices 19 to 30 in Python (0-based indexing)
columns_of_interest = df.columns[19:31]  # Select columns T to AE

# Count the number of rows where columns T to AE are not all 'n/a'
# We will treat both 'n/a' and NaN as equivalent for this purpose
count_non_na_rows = df[columns_of_interest].replace('n/a', pd.NA).dropna(how='all').shape[0]

count_non_na_rows


# Remove the rows where columns_of_interest are all null values and save this in a new Excel file
df = df[~df[columns_of_interest].replace('n/a', pd.NA).isnull().all(axis=1)]

output_file_path = 'interopSurvey.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Output saved to: {output_file_path}")



In [ ]:
# Open interopSurvey.xlsx
df = pd.read_excel('interopSurvey.xlsx')

# For each column, count the number of rows for each unique value. This is like a sets of unique values in each column and their counts
unique_values = {col: df[col].value_counts() for col in df.columns}


In [ ]:
# count the categories in Q67. Some of them has this format "category1, category2, category3"
# We will split them and count the number of each category
categories = df['Q67'].str.split(',').explode().str.strip().value_counts()

categories



In [ ]:
# Count how many df['Q65'] are not null
Q65_sum = df['Q65'].notnull().sum()
print(f"Total number of non-null values in column Q65: {Q65_sum}")

# Define a placeholder that does not appear in your data
placeholder = "eg"

# Replace 'e.g.,' with the placeholder, split, explode, and then replace the placeholder back
ONNX_bug_categories = (
    df['Q65']
    .str.replace('e.g.,', placeholder, regex=False)  # Temporarily replace 'e.g.,' with a placeholder
    .str.split(',')  # Split on commas
    .explode()  # Convert lists to rows
    .str.strip()  # Strip whitespace
    .str.replace(placeholder, 'e.g.,', regex=False)  # Replace the placeholder back to 'e.g.,'
    .value_counts()  # Count occurrences
)

# Calculate the percentage of each category (int)
ONNX_bug_categories_percentage = round(ONNX_bug_categories / Q65_sum * 100)

# Combine ONNX_bug_categories and ONNX_bug_categories_percentage into a DataFrame
ONNX_bug_summary = pd.DataFrame({'Count': ONNX_bug_categories, 'Percentage': ONNX_bug_categories_percentage})



# Print     df['Q65_3_TEXT'] value which is not null
print(f"Number of non-null values in column Q65_3_TEXT: {df['Q65_3_TEXT'].notnull().sum()}")
print(df['Q65_3_TEXT'].replace('n/a', pd.NA).dropna().unique())


ONNX_bug_summary



In [ ]:
# print out the first row in df.columns[21]
print(f"Column 21: {df[df.columns[21]].iloc[0]}")

In [ ]:
# Same analysis for Q63
Q63_sum = df['Q63'].notnull().sum()
print(f"Total number of non-null values in column Q63: {Q63_sum}")

placeholder = "eg"

usage_categories = (
    df['Q63']
    .str.replace('e.g.,', placeholder, regex=False)  # Temporarily replace 'e.g.,' with a placeholder
    .str.split(',')  # Split on commas
    .explode()  # Convert lists to rows
    .str.strip()  # Strip whitespace
    .value_counts()  # Count occurrences
)

usage_categories_percentage = round(usage_categories / Q63_sum * 100)

usage_summary = pd.DataFrame({'Count': usage_categories, 'Percentage': usage_categories_percentage})

usage_summary